In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2


# ------------------------------
# PARAMETERS & PATHS
# ------------------------------
# Path to your saved teacher classifier model (in native Keras format)
classifier_model_path = "teacher_classifier_modelv5.keras"
# Name of the target convolutional layer to use for Grad-CAM.
# IMPORTANT: Make sure this exactly matches one of the layer names printed by model.summary().
TARGET_LAYER_NAME = "block4a_expand_activation"  # (adjust if necessary)


# ------------------------------
# IMAGE PREPROCESSING FUNCTION
# ------------------------------
def preprocess_image(img_path, target_size=(224, 224)):
   """
   Loads and preprocesses an image.
   """
   img = cv2.imread(img_path)
   if img is None:
       raise ValueError(f"Image not found at path: {img_path}")
   img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
   img = cv2.resize(img, target_size)
   img = img.astype(np.float32) / 255.0  # Normalize to [0,1]
   return np.expand_dims(img, axis=0)  # Add batch dimension


# ------------------------------
# GRAD-CAM FUNCTION
# ------------------------------
def make_gradcam_heatmap(img_array, model, target_layer_name, pred_index=None):
   """
   Generates a Grad-CAM heatmap for a given image and model.
  
   Args:
       img_array (np.array): Preprocessed image array of shape (1, height, width, channels)
       model (tf.keras.Model): The trained classifier model.
       target_layer_name (str): The name of the target convolutional layer.
       pred_index (int, optional): Index of the target class. If None, uses the model's prediction.
  
   Returns:
       heatmap (np.array): 2D heatmap (values between 0 and 1).
   """
   # Create a model mapping the input image to the activations of the target layer and the final predictions.
   try:
       target_layer = model.get_layer(target_layer_name)
   except ValueError:
       raise ValueError(f"No such layer: {target_layer_name}. "
                        f"Available layers are: {[layer.name for layer in model.layers]}")
  
   grad_model = tf.keras.models.Model(
       [model.inputs],
       [target_layer.output, model.output]
   )
  
   with tf.GradientTape() as tape:
       conv_outputs, predictions = grad_model(img_array)
       if pred_index is None:
           pred_index = tf.argmax(predictions[0])
       class_channel = predictions[:, pred_index]
  
   # Compute gradients of the class score with respect to the output feature map.
   grads = tape.gradient(class_channel, conv_outputs)
  
   # Compute the channel-wise mean of the gradients.
   pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
  
   conv_outputs = conv_outputs[0]
   # Multiply each channel by its corresponding weight.
   heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
   heatmap = tf.squeeze(heatmap)
  
   # Normalize the heatmap to have values between 0 and 1.
   heatmap = tf.maximum(heatmap, 0) / (tf.reduce_max(heatmap) + 1e-10)
   return heatmap.numpy()


# ------------------------------
# UTILITY FUNCTION TO OVERLAY HEATMAP
# ------------------------------
def overlay_heatmap(heatmap, image, alpha=0.4, colormap=cv2.COLORMAP_JET):
   """
   Overlays the Grad-CAM heatmap onto the original image.
  
   Args:
       heatmap (np.array): 2D heatmap.
       image (np.array): Original image in RGB (without batch dimension).
       alpha (float): Transparency factor.
       colormap: OpenCV colormap to apply.
  
   Returns:
       output (np.array): Image with heatmap overlay.
   """
   # Resize heatmap to match the image size.
   heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
   heatmap = np.uint8(255 * heatmap)
   heatmap = cv2.applyColorMap(heatmap, colormap)
  
   # Convert original image to BGR for blending.
   image_bgr = cv2.cvtColor(np.uint8(255 * image), cv2.COLOR_RGB2BGR)
   overlayed = cv2.addWeighted(heatmap, alpha, image_bgr, 1 - alpha, 0)
   output = cv2.cvtColor(overlayed, cv2.COLOR_BGR2RGB)
   return output


# ------------------------------
# MAIN SCRIPT
# ------------------------------
if __name__ == "__main__":
   # Load the classifier model.
   print("Loading classifier model...")
   classifier = tf.keras.models.load_model(classifier_model_path)
  
   # Specify the path to a test image.
   test_img_path = "/Users/morgan/Desktop/BT-ClassificationDissCode/LabelledFigshare/1/11.jpg"
   #/Users/morgan/Desktop/BT-ClassificationDissCode/LabelledFigshare/1/11.jpg
  
   #/Users/morgan/Desktop/BT-ClassificationDissCode/LabelledFigshare/2/726.jpg
  
   # Preprocess the test image.
   img_array = preprocess_image(test_img_path)
  
   # Generate the Grad-CAM heatmap.
   heatmap = make_gradcam_heatmap(img_array, classifier, TARGET_LAYER_NAME)
  
   # Load the original image (for overlay).
   original_img = cv2.cvtColor(cv2.imread(test_img_path), cv2.COLOR_BGR2RGB)
   original_img = cv2.resize(original_img, (224, 224))
  
   # Overlay the heatmap on the original image.
   overlayed_img = overlay_heatmap(heatmap, original_img)
  
   # Plot the original image, the raw heatmap, and the overlayed image.
   plt.figure(figsize=(12, 4))
   plt.subplot(1, 3, 1)
   plt.imshow(original_img)
   plt.title("Original Image")
   plt.axis("off")
  
   plt.subplot(1, 3, 2)
   plt.imshow(heatmap, cmap="jet")
   plt.title("Grad-CAM Heatmap")
   plt.axis("off")
  
   plt.subplot(1, 3, 3)
   plt.imshow(overlayed_img)
   plt.title("Overlayed Grad-CAM")
   plt.axis("off")
  
   plt.tight_layout()
   plt.show()
